# Problem Description

An ant is sitting on an infinite grid of white and black squares. Initially, the grid is all white and the ant faces right. At each step, it does the following:
1. At a white square, flip the color of the square, turn 90 degree right (clockwise), and move forward one unit.
2. At a black sqaure, flip the color of the square, turn 90 degree left (counter-clockwise), andmove forward one unit.

Write a program to simulate the first K moves that the ant makes and print the final board as a grid.
Note that you are not provided with the data structure to represent the grid. This is something you must design yourself. The only input to your method is K. You should print the final grid and return
nothing. The method signature might be something like void PrintKMoves(int K).

### Summary

The code simulates the movement of an ant on an infinite grid based on specific rules:
   * If the ant is on a white square, it flips the color to black, turns 90 degrees right, and moves forward one unit.
   * If the ant is on a black square, it flips the color to white, turns 90 degrees left, and moves forward one unit.

This code sets up an infinite grid using a dynamically growing 2D list. It simulates the movements of the ant according to the rules and prints the final state of the grid after K moves.

### Position Class

* Purpose: Represents the ant's position on the grid with row and column coordinates.
* Methods:
   * equals(other): Checks if two positions are equal.
   * hash_code(): Generates a hash code for the position.
   * clone(): Creates a copy of the position.

In [2]:
class Position:
    '''
    Position Class
        Position(row: int, column: int): Represents a position on the grid with row and column coordinates.
        Equals(other: 'Position') -> bool: Checks if two positions are equal.
        hash_code() -> int: Generates a hash code for the position, used for comparisons.
        clone() -> 'Position': Creates a copy of the position.
    '''
    def __init__(self, row: int, column: int):
        self.row: int = row
        self.column: int = column

    def equals(self, other: 'Position') -> bool:
        return self.row == other.row and self.column == other.column

    def hash_code(self) -> int:
        return (self.row * 31) ^ self.column

    def clone(self) -> 'Position':
        return Position(self.row, self.column)

### OrientationEnum Enum

Defines the possible directions the ant can face: LEFT, UP, RIGHT, and DOWN.

In [3]:
from enum import Enum

class OrientationEnum(Enum):
    '''
    OrientationEnum Enum
        Defines the possible orientations of the ant: LEFT, UP, RIGHT, and DOWN.
    '''
    LEFT = 1
    UP = 2
    RIGHT = 3
    DOWN = 4

### Orientation Class

* Purpose: Represents the current direction the ant is facing.
* Methods:
   * get_turn(clockwise): Determines the new direction based on a clockwise or counterclockwise turn.

In [4]:
class Orientation:
    '''
    Orientation Class
        Orientation(orientation: OrientationEnum): Represents the current orientation of the ant.
        get_turn(clockwise: bool) -> OrientationEnum: Determines the new orientation based on a clockwise or counterclockwise turn.
    '''
    def __init__(self, orientation: OrientationEnum):
        self.orient: OrientationEnum = orientation

    def get_turn(self, clockwise: bool) -> OrientationEnum:
        if self.orient == OrientationEnum.LEFT:
            return OrientationEnum.UP if clockwise else OrientationEnum.DOWN
        elif self.orient == OrientationEnum.UP:
            return OrientationEnum.RIGHT if clockwise else OrientationEnum.LEFT
        elif self.orient == OrientationEnum.RIGHT:
            return OrientationEnum.DOWN if clockwise else OrientationEnum.UP
        else:
            return OrientationEnum.LEFT if clockwise else OrientationEnum.RIGHT

### Ant Class

* Purpose: Represents the ant with its position and direction.
* Methods:
   * turn(clockwise): Turns the ant 90 degrees clockwise or counterclockwise.
   * move(): Moves the ant one unit forward in the current direction.
   * adjust_position(shift_row, shift_column): Adjusts the ant's position when the grid expands.

In [5]:
class Ant:
    '''
    Ant Class
        Ant(): Initializes the ant with a starting position (0, 0) and facing RIGHT.
        turn(clockwise: bool) -> None: Turns the ant 90 degrees clockwise or counterclockwise.
        move() -> None: Moves the ant one unit forward in the current direction.
        adjust_position(shift_row: int, shift_column: int) -> None: Adjusts the position of the ant when the grid is expanded.
    '''
    def __init__(self):
        self.position: Position = Position(0, 0)
        self.orientation: Orientation = Orientation(OrientationEnum.RIGHT)

    def turn(self, clockwise: bool) -> None:
        self.orientation.orient = self.orientation.get_turn(clockwise)

    def move(self) -> None:
        if self.orientation.orient == OrientationEnum.LEFT:
            self.position.column -= 1
        elif self.orientation.orient == OrientationEnum.RIGHT:
            self.position.column += 1
        elif self.orientation.orient == OrientationEnum.UP:
            self.position.row -= 1
        elif self.orientation.orient == OrientationEnum.DOWN:
            self.position.row += 1

    def adjust_position(self, shift_row: int, shift_column: int) -> None:
        self.position.row += shift_row
        self.position.column += shift_column

### Grid Class

* Purpose: Represents the infinite grid and manages the ant's movements.
* Methods:
   * copy_with_shift(old_grid, new_grid, shift_row, shift_column): Copies the old grid to a new grid with a specified shift.
   * ensure_fit(position): Expands the grid if necessary to accommodate the ant's new position.
   * flip(position): Flips the color of the cell at the given position.
   * move(): Performs the ant's move (turn, flip, move forward) and ensures the grid can fit the new position.
   * print_grid(): Prints the current state of the grid, where 'X' represents a black cell and '_' represents a white cell.

In [6]:
class Grid:
    '''
    This program sets up an infinite grid using a dynamic 2D list. It simulates the movements of the ant according to the rules and
    prints the final state of the grid after K moves.
    
    Grid Class
        Grid(): Initializes the grid and the ant.
        copy_with_shift(old_grid: list[list[bool]], new_grid: list[list[bool]], shift_row: int, shift_column: int) -> None: Copies the old grid to a new grid with a specified shift.
        ensure_fit(position: Position) -> None: Ensures that the grid can accommodate the ant's new position by expanding it if necessary.
        flip(position: Position) -> None: Flips the color of the cell at the given position.
        move() -> None: Turns, flips, and moves the ant. It also ensures the grid is large enough to accommodate the ant's new position.
        print_grid() -> None: Prints the current state of the grid, where 'X' represents a black cell and '_' represents a white cell.
    '''
    def __init__(self):
        self.grid: list[list[bool]] = [[False]]
        self.ant: Ant = Ant()

    def copy_with_shift(self, old_grid: list[list[bool]], new_grid: list[list[bool]], shift_row: int, shift_column: int) -> None:
        for r in range(len(old_grid)):
            for c in range(len(old_grid[0])):
                new_grid[r + shift_row][c + shift_column] = old_grid[r][c]

    def ensure_fit(self, position: Position) -> None:
        shift_row: int = 0
        shift_column: int = 0
        num_rows: int = len(self.grid)
        if position.row < 0:
            shift_row = num_rows
            num_rows += num_rows
        elif position.row >= num_rows:
            num_rows *= 2

        num_columns: int = len(self.grid[0])
        if position.column < 0:
            shift_column = num_columns
            num_columns += num_columns
        elif position.column >= num_columns:
            num_columns *= 2

        if num_rows != len(self.grid) or num_columns != len(self.grid[0]):
            new_grid: list[list[bool]] = [[False for _ in range(num_columns)] for _ in range(num_rows)]
            self.copy_with_shift(self.grid, new_grid, shift_row, shift_column)
            self.ant.adjust_position(shift_row, shift_column)
            self.grid = new_grid

    def flip(self, position: Position) -> None:
        row: int = position.row
        column: int = position.column
        self.grid[row][column] = not self.grid[row][column]

    def move(self):
        self.ant.turn(not self.grid[self.ant.position.row][self.ant.position.column])
        self.flip(self.ant.position)
        self.ant.move()
        self.ensure_fit(self.ant.position)

    def print_grid(self) -> None:
        top_left: Position = Position(0, 0)
        bottom_right: Position = Position(len(self.grid) - 1, len(self.grid[0]) - 1)
        for r in range(top_left.row, bottom_right.row + 1):
            line = ''
            for c in range(top_left.column, bottom_right.column + 1):
                line += 'X' if self.grid[r][c] else '_'
            print(line)

## Example Usage

Here's how you can use these methods to find the pair of values:

In [7]:
def PrintKMovesGrid(K: int) -> None:
    grid = Grid()
    for _ in range(K):
        grid.move()
    grid.print_grid()


# Example usage:
# In this example, PrintKMoves(10) will simulate 10 moves of the ant and print the grid.
PrintKMovesGrid(10)

____
__X_
XXX_
XX__


# Literature

The contents base on the following literature:

* Gayle Laakmann McDowell, *Cracking the Coding Interview*, [Link](https://www.crackingthecodinginterview.com/).

**Copyright**

The notebooks are provided as [Open Educational Resources](https://en.wikipedia.org/wiki/Open_educational_resources). Feel free to use the notebooks for your own purposes. The text is licensed under [Creative Commons Attribution 4.0](https://creativecommons.org/licenses/by/4.0/), the code of the IPython examples under the [MIT license](https://opensource.org/licenses/MIT).